In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/platesv2/sample_submission.csv
/kaggle/input/platesv2/plates.zip


In [2]:
import zipfile

zip_path = "/kaggle/input/platesv2/plates.zip"
extract_path = "/kaggle/working/plates"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Extracted files:", os.listdir(extract_path))


Extracted files: ['plates', '__MACOSX']


In [3]:
train_dir = os.listdir("/kaggle/working/plates/plates/train")
test_dir = os.listdir("/kaggle/working/plates/plates/test")

train_dir

['cleaned', '.DS_Store', 'dirty']

In [4]:
from torchvision import datasets, transforms

# Define the correct path to the train folder
train_dir = "/kaggle/working/plates/plates/train"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize all images
    transforms.ToTensor()            # Convert to Tensor
])

# Load the training dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# Print classes to verify
print("Classes:", train_dataset.classes)


Classes: ['cleaned', 'dirty']


In [5]:
from torch.utils.data import DataLoader

# Create DataLoader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Check some sample data
data_iter = iter(train_loader)
images, labels = next(data_iter)
print("Batch of images:", images.shape)
print("Batch of labels:", labels)


Batch of images: torch.Size([4, 3, 224, 224])
Batch of labels: tensor([0, 0, 1, 0])


In [6]:
import torch
from torchvision import models

# Load ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final layer for 2 classes (dirty and clean)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)  # Output size = 2

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 201MB/s]


In [8]:
import torch.optim as optim

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [10]:
# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/50, Loss: 0.6352248787879944
Epoch 2/50, Loss: 0.5107301563024521
Epoch 3/50, Loss: 0.5132587671279907
Epoch 4/50, Loss: 0.4414399921894073
Epoch 5/50, Loss: 0.4324199497699738
Epoch 6/50, Loss: 0.40569522976875305
Epoch 7/50, Loss: 0.4118296250700951
Epoch 8/50, Loss: 0.452118257433176
Epoch 9/50, Loss: 0.31927633583545684
Epoch 10/50, Loss: 0.3165430337190628
Epoch 11/50, Loss: 0.33702687323093417
Epoch 12/50, Loss: 0.3507377587258816
Epoch 13/50, Loss: 0.3134429931640625
Epoch 14/50, Loss: 0.34882262647151946
Epoch 15/50, Loss: 0.32602500542998314
Epoch 16/50, Loss: 0.34604691416025163
Epoch 17/50, Loss: 0.2711860820651054
Epoch 18/50, Loss: 0.2708060421049595
Epoch 19/50, Loss: 0.3262872636318207
Epoch 20/50, Loss: 0.22157092541456222
Epoch 21/50, Loss: 0.21189157366752626
Epoch 22/50, Loss: 0.23923047557473182
Epoch 23/50, Loss: 0.2114149495959282
Epoch 24/50, Loss: 0.2825522720813751
Epoch 25/50, Loss: 0.31170803755521775
Epoch 26/50, Loss: 0.5270017474889755
Epoch 27/50,

In [14]:
from torchvision import transforms
from PIL import Image
import os

# Define the test folder path
test_dir = "/kaggle/working/plates/plates/test"

# Define transformations (same as train)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load test images manually
test_images = []
test_ids = []

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    image = Image.open(img_path).convert("RGB")  # Ensure 3 channels (RGB)
    image = transform(image)  # Apply transformations
    test_images.append(image)
    test_ids.append(img_name.split('.')[0])  # Extract ID from filename

print(f"Loaded {len(test_images)} test images.")

Loaded 744 test images.


In [21]:
from torchvision import transforms
from PIL import Image
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

# Define the test folder path
test_dir = "/kaggle/working/plates/plates/test"

# Define transformations (same as train)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load test images manually
test_images = []
test_ids = []

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    
    # Skip non-image files like .DS_Store
    if img_name.startswith('.') or not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue
    
    image = Image.open(img_path).convert("RGB")  # Ensure 3 channels (RGB)
    image = transform(image)  # Apply transformations
    test_images.append(image)
    test_ids.append(img_name.split('.')[0])  # Extract ID from filename

print(f"Loaded {len(test_images)} test images.")

# Convert list to tensor
test_images = torch.stack(test_images)

# Create a TensorDataset for test data
test_dataset = TensorDataset(test_images)

# Create a DataLoader for batching (batch_size 8)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Set model to evaluation mode
model.eval()

# List to store predictions
predictions = []

# Inference loop for test dataset
with torch.no_grad():  # No need to track gradients during inference
    for images in test_loader:
        images = images[0].to(device)  # Images are wrapped in a tuple, so access images[0]
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

# Convert predictions to 'clean' or 'dirty' labels
predicted_labels = ['clean' if pred == 0 else 'dirty' for pred in predictions]

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'id': test_ids,
    'label': predicted_labels
})

# Save the predictions to CSV
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission saved!")


Loaded 744 test images.
Submission saved!


In [22]:
############   ABOVE CODE ACCURACY IS ONLY 59  ###########

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from PIL import Image
import os
import pandas as pd


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),   # Data augmentation
    transforms.RandomRotation(30),       # Data augmentation
    transforms.RandomAffine(15),         # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

# Define transformations for test (no augmentation, only resizing)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [26]:
# Define train directory
train_dir = "/kaggle/working/plates/plates/train"

# Load training data
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# Split dataset into training and validation sets (80% training, 20% validation)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader for training, validation, and test
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Define test directory and test data
test_dir = "/kaggle/working/plates/plates/test"
test_images = []
test_ids = []

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    try:
        image = Image.open(img_path).convert("RGB")
        image = test_transform(image)  # Apply transformations
        test_images.append(image)
        test_ids.append(img_name.split('.')[0])  # Extract ID from filename
    except:
        continue

# Create a DataLoader for test dataset
test_images_tensor = torch.stack(test_images)


In [27]:
# Load pre-trained ResNet18 model and modify the final layer for 2 classes
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust for 2 classes (clean, dirty)

# Move model to device
model = model.to(device)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Learning rate scheduler (decreases LR after 7 epochs)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [29]:
# Function to train the model
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=10):
    best_acc = 0.0  # To keep track of the best accuracy during training
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Zero gradients, perform a backward pass, and update the weights
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

            running_loss += loss.item()

        # Print statistics for each epoch
        train_loss = running_loss / len(train_loader)
        train_acc = correct_preds / total_preds
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_correct_preds = 0
        val_total_preds = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct_preds += torch.sum(preds == labels).item()
                val_total_preds += labels.size(0)

        val_acc = val_correct_preds / val_total_preds
        print(f"Validation Accuracy: {val_acc:.4f}")

        # Save the best model based on validation accuracy
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        scheduler.step()  # Step the learning rate scheduler

    print(f"Best Validation Accuracy: {best_acc:.4f}")


In [31]:
# Train the model
train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=100)


Epoch 1/100, Loss: 0.4124, Accuracy: 0.8438
Validation Accuracy: 0.6250
Epoch 2/100, Loss: 0.3377, Accuracy: 0.8750
Validation Accuracy: 0.6250
Epoch 3/100, Loss: 0.3791, Accuracy: 0.9062
Validation Accuracy: 0.6250
Epoch 4/100, Loss: 0.3419, Accuracy: 0.9062
Validation Accuracy: 0.6250
Epoch 5/100, Loss: 0.1942, Accuracy: 0.9688
Validation Accuracy: 0.6250
Epoch 6/100, Loss: 0.2656, Accuracy: 0.9062
Validation Accuracy: 0.6250
Epoch 7/100, Loss: 0.2738, Accuracy: 0.8750
Validation Accuracy: 0.6250
Epoch 8/100, Loss: 0.3552, Accuracy: 0.8438
Validation Accuracy: 0.6250
Epoch 9/100, Loss: 0.2258, Accuracy: 0.9062
Validation Accuracy: 0.6250
Epoch 10/100, Loss: 0.4306, Accuracy: 0.8438
Validation Accuracy: 0.6250
Epoch 11/100, Loss: 0.2797, Accuracy: 0.8125
Validation Accuracy: 0.6250
Epoch 12/100, Loss: 0.2837, Accuracy: 0.8125
Validation Accuracy: 0.6250
Epoch 13/100, Loss: 0.2313, Accuracy: 0.9688
Validation Accuracy: 0.5000
Epoch 14/100, Loss: 0.2501, Accuracy: 0.9062
Validation Accu

In [32]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Set the model to evaluation mode
model.eval()

# Inference on test images
test_preds = []
with torch.no_grad():
    for image in test_images_tensor:
        image = image.unsqueeze(0).to(device)  # Add batch dimension
        output = model(image)
        _, pred = torch.max(output, 1)
        test_preds.append(pred.item())

# Convert predictions to labels
predicted_labels = ['clean' if pred == 0 else 'dirty' for pred in test_preds]

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': test_ids,
    'label': predicted_labels
})

# Save the predictions to CSV
submission_df.to_csv('/kaggle/working/submission_plates.csv', index=False)
print("Submission saved!")


/tmp/ipykernel_23/3064993140.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Submission saved!


In [33]:
###############  ABOVE CODE'S ACCURACY IS TOO WORST - 21 ###################

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from PIL import Image
import os
import pandas as pd

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations (augmentation for training and normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit the model
    transforms.RandomHorizontalFlip(),   # Augment the data by flipping
    transforms.RandomRotation(30),       # Augment the data by rotating
    transforms.RandomAffine(15),         # Augment the data by affine transformation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with mean & std
])

# Test data transformations (no augmentation, only resizing)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define train directory and load dataset
train_dir = "/kaggle/working/plates/plates/train"
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader for training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust for 2 classes (clean, dirty)
model = model.to(device)  # Move model to GPU or CPU

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Learning rate scheduler (optional for better optimization)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Function to train the model
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=10):
    best_acc = 0.0  # Best accuracy to track best model
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Zero gradients, perform a backward pass, and update the weights
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

            running_loss += loss.item()

        # Print statistics for each epoch
        train_loss = running_loss / len(train_loader)
        train_acc = correct_preds / total_preds
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_correct_preds = 0
        val_total_preds = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct_preds += torch.sum(preds == labels).item()
                val_total_preds += labels.size(0)

        val_acc = val_correct_preds / val_total_preds
        print(f"Validation Accuracy: {val_acc:.4f}")

        # Save the best model based on validation accuracy
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        scheduler.step()  # Step the learning rate scheduler

    print(f"Best Validation Accuracy: {best_acc:.4f}")

# Train the model
train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=50)


Epoch 1/50, Loss: 0.7863, Accuracy: 0.6875
Validation Accuracy: 0.5000
Epoch 2/50, Loss: 0.8317, Accuracy: 0.8125
Validation Accuracy: 0.3750
Epoch 3/50, Loss: 1.1579, Accuracy: 0.7500
Validation Accuracy: 0.1250
Epoch 4/50, Loss: 0.5298, Accuracy: 0.7812
Validation Accuracy: 0.6250
Epoch 5/50, Loss: 0.1951, Accuracy: 0.9062
Validation Accuracy: 0.7500
Epoch 6/50, Loss: 0.1358, Accuracy: 0.9375
Validation Accuracy: 0.5000
Epoch 7/50, Loss: 0.1809, Accuracy: 0.9375
Validation Accuracy: 0.5000
Epoch 8/50, Loss: 0.2208, Accuracy: 0.9062
Validation Accuracy: 0.5000
Epoch 9/50, Loss: 0.1597, Accuracy: 0.9375
Validation Accuracy: 0.5000
Epoch 10/50, Loss: 0.0891, Accuracy: 0.9688
Validation Accuracy: 0.5000
Epoch 11/50, Loss: 0.0671, Accuracy: 1.0000
Validation Accuracy: 0.5000
Epoch 12/50, Loss: 0.1259, Accuracy: 0.9375
Validation Accuracy: 0.6250
Epoch 13/50, Loss: 0.0736, Accuracy: 1.0000
Validation Accuracy: 0.6250
Epoch 14/50, Loss: 0.1430, Accuracy: 0.9688
Validation Accuracy: 0.6250
E

In [36]:
# Load the best model for inference
model.load_state_dict(torch.load('best_model.pth'))

# Set model to evaluation mode
model.eval()

# Define test directory
test_dir = "/kaggle/working/plates/plates/test"
test_images = []
test_ids = []

# Preprocess the test data
for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    try:
        image = Image.open(img_path).convert("RGB")
        image = test_transform(image)
        test_images.append(image)
        test_ids.append(img_name.split('.')[0])  # Extract ID from filename
    except:
        continue

# Convert to tensor
test_images_tensor = torch.stack(test_images)

# Inference on test data
test_preds = []
with torch.no_grad():
    for image in test_images_tensor:
        image = image.unsqueeze(0).to(device)  # Add batch dimension
        output = model(image)
        _, pred = torch.max(output, 1)
        test_preds.append(pred.item())

# Convert predictions to 'clean' or 'dirty' labels
predicted_labels = ['clean' if pred == 0 else 'dirty' for pred in test_preds]

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': test_ids,
    'label': predicted_labels
})

# Save the predictions to CSV
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission saved!")


/tmp/ipykernel_23/4266489762.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Submission saved!
